In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/home/antoine/homhots/HOTS/HOTS


In [2]:
import time
def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')

# loading raw dataset

In [3]:
import numpy as np
import tonic
download = False
learn_set = tonic.datasets.NMNIST(save_to='../Data/',
                                  train=True, download=download,
                                  transform=tonic.transforms.AERtoVector()
                                 )
nb_pola = 2

In [4]:
records_path = '../Records'
timestr = '2021-02-16'

ds = 12
ds = 1200
nb_train = int(len(learn_set)//ds)
print(f'The dataset has size {len(learn_set)}, using {nb_train}')

The dataset has size 60000, using 50


# loading output of HOTS as a dataset

# fitting LR on the output of HOTS

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type("torch.DoubleTensor") # -> torch.tensor([1.2, 3]).dtype = torch.float64
# https://sebastianraschka.com/faq/docs/pytorch-crossentropy.html
#criterion = torch.nn.NLLLoss(reduction="mean") # loss divided by output size
criterion = torch.nn.BCELoss(reduction="mean") # loss divided by output size

class LogisticRegressionModel(torch.nn.Module):
    #torch.nn.Module -> Base class for all neural network modules
    def __init__(self, N, n_classes, bias=True):
        super(LogisticRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(N, n_classes, bias=bias)
        self.nl = torch.nn.Softmax(dim=1)

    def forward(self, factors):
        return self.nl(self.linear(factors))

In [6]:
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 5 + 1
#num_epochs = 2 ** 9 + 1
num_epochs

33

In [7]:
# batch_size = 256
n_classes=10
amsgrad = False # gives similar results
amsgrad = True  # gives similar results

def fit_data(dataset, 
            nb_digit,
            nb_pola,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            betas=betas,
            verbose=False, #**kwargs
        ): 

    generator = torch.Generator().manual_seed(42)
    sampler = torch.utils.data.RandomSampler(dataset, replacement=True, num_samples=nb_digit, generator=generator)
    loader = tonic.datasets.DataLoader(dataset, sampler=sampler)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'device -> {device}')

    N = dataset.sensor_size[0]*dataset.sensor_size[1]*nb_pola
    n_classes = len(dataset.classes)
    logistic_model = LogisticRegressionModel(N, n_classes)
    print(N, n_classes)
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )

    for epoch in range(int(num_epochs)):
        losses = []
        for X, label in loader:
            X, label = X.to(device), label.to(device)
            X, label = X.squeeze(0), label.squeeze(0) # just one digit = one batch
        
            outputs = logistic_model(X)

            n_events = X.shape[0]
            labels = label*torch.ones(n_events).type(torch.LongTensor).to(device)
            labels = torch.nn.functional.one_hot(labels, num_classes=n_classes).type(torch.DoubleTensor).to(device)

            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")
            
    return logistic_model, losses

In [8]:
import pickle
tic()
model, loss = fit_data(learn_set, 
            nb_train,
            nb_pola,           
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            betas=betas,
            verbose=True,
        )
f_name = records_path+'/EXP_03_NMNIST/'+timestr+f'_LR_raw_{nb_train}.pkl'
with open(f_name, 'wb') as file:
    pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)
toc()

device -> cuda
2312 10
Iteration: 0 - Loss: 0.50124
Iteration: 1 - Loss: 0.33822
Iteration: 2 - Loss: 0.28222
Iteration: 3 - Loss: 0.41433
Iteration: 4 - Loss: 0.37538
Iteration: 5 - Loss: 0.25121
Iteration: 6 - Loss: 0.26826
Iteration: 7 - Loss: 0.14865
Iteration: 8 - Loss: 0.30238
Iteration: 9 - Loss: 0.22289
Iteration: 10 - Loss: 0.20861
Iteration: 11 - Loss: 0.28110
Iteration: 12 - Loss: 0.21364
Iteration: 13 - Loss: 0.37234
Iteration: 14 - Loss: 0.41418
Iteration: 15 - Loss: 0.32171
Iteration: 16 - Loss: 0.18329
Iteration: 17 - Loss: 0.18516
Iteration: 18 - Loss: 0.37263
Iteration: 19 - Loss: 0.20245
Iteration: 20 - Loss: 0.19848
Iteration: 21 - Loss: 0.19865
Iteration: 22 - Loss: 0.30225
Iteration: 23 - Loss: 0.35352
Iteration: 24 - Loss: 0.25289
Iteration: 25 - Loss: 0.29442
Iteration: 26 - Loss: 0.26507
Iteration: 27 - Loss: 0.30226
Iteration: 28 - Loss: 0.20938
Iteration: 29 - Loss: 0.18553
Iteration: 30 - Loss: 0.25738
Iteration: 31 - Loss: 0.34651
Iteration: 32 - Loss: 0.278

# testing output of HOTS as a dataset

In [9]:
test_set = tonic.datasets.NMNIST(save_to='../Data/',
                                train=False, download=download,
                                transform=tonic.transforms.AERtoVector())

In [10]:
nb_test = int(len(test_set)//ds)
print(f'The dataset has size {len(test_set)}, using {nb_test}')

The dataset has size 10000, using 8


In [11]:
def predict_data(test_set, model, 
            verbose=False, **kwargs
        ):
    
    with torch.no_grad():

        generator=torch.Generator().manual_seed(42)
        sampler = torch.utils.data.RandomSampler(test_set, replacement=True, num_samples=nb_test, generator=generator)
        loader = tonic.datasets.DataLoader(test_set, sampler=sampler)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        logistic_model = model.to(device)

        pred_target, true_target = [], []

        for X, label in loader:
            X = X.to(device)
            X, label = X.squeeze(0), label.squeeze(0)

            n_events = X.shape[0]
            labels = label*torch.ones(n_events).type(torch.LongTensor)

            outputs = logistic_model(X)

            pred_target.append(torch.argmax(outputs, dim=1).cpu().numpy())
            true_target.append(labels.numpy())

    return pred_target, true_target

In [12]:
pred_target, true_target = predict_data(test_set, model)

In [13]:
accuracy = []
for pred_target_, true_target_ in zip(pred_target, true_target):
    accuracy.append(np.mean(pred_target_ == true_target_))
print(f'{np.mean(accuracy)=:.3f}')

np.mean(accuracy)=0.809
